# Blue Club Functions

In [1]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np
import scipy.stats as stats
from endplay import *

In [2]:
def points_rs(hnd):
   long = 0
   short = dist_points(hnd, [2, 1, 0, 0])
   L1 = dist_points(hnd, [0, 0, 0, 0, 0, 1])
   L2 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 2])
   L3 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 0, 3])
   L4 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 0, 0, 4])
   L5 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 0, 0, 0, 5])
   L6 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6])
   L7 = dist_points(hnd, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7])
   long = max(L1, L2, L3, L4, L5, L6, L7)
   if (shape(hnd) == [5,5,3,0] or shape(hnd) == [5,5,2,1]):
      long = long +1
   if (shape(hnd) == [6,5,2,0] or shape(hnd) == [6,5,1,1]):
      long = long +1
   if (shape(hnd) == [6,6,1,0]):
      long = long +2
   if (shape(hnd) == [7,5,1,0]):
      long = long +1
   if (shape(hnd) == [7,6,0,0]):
      long = long +2
   if (shape(hnd) == [8,5,0,0]):
      long = long +1
   totalPts = short + long + hcp(hnd)
   return totalPts

In [3]:
def rule20(h):
   TF = False
   if (hcp(h) + shape(h)[0] + shape(h)[1]) >= 20:
      TF = True
   return TF

In [4]:
def bid_qual(soot):
   if (len(soot) == 0):
      eval = 0 ;
   else:
      eval = len(soot) + ( hcp(soot) / 10 )
   return eval

In [5]:
def quals(h):
   sq = bid_qual(h.spades) ; hq = bid_qual(h.hearts) ; dq = bid_qual(h.diamonds) ; cq = bid_qual(h.clubs) ;
   q = make_array(sq, hq, dq, cq)
   return q

In [6]:
def control(soot):
   length = len(soot)
   control = 0
   if (length == 0):
      control = 1
   if (length == 1 and hcp(soot) == 0):
      control = 2
   if (length == 1 and soot.__contains__('A')):
      control = 1
   if (length == 2 and soot.__contains__('A')):
      control = 1
   if (length == 2 and hcp(soot) == 3 and soot.__contains__('K')):
      control = 2
   return control

In [7]:
def make_bid_table(hnd):
   control_array = make_array( control(hnd.spades) , control(hnd.hearts) , control(hnd.diamonds) , control(hnd.clubs) )
   array_hcp = make_array(hcp(hnd.spades),hcp(hnd.hearts),hcp(hnd.diamonds),hcp(hnd.clubs))
   array_length = exact_shape(hnd)
   array_titles = make_array('Spades','Hearts','Diamonds','Clubs')
   q = quals(hnd)
   bid_tab = Table().with_columns('Suit', array_titles, 'Length', array_length, 'HCP', array_hcp, 'Controls', control_array, 'Bid Quality', q)
   return bid_tab

In [8]:
def balanced(hnd):
   ball = False
   if ( shape(hnd) == [4,3,3,3] ):
      ball = True
   elif ( shape(hnd) == [4,4,3,2] ):
      ball = True
   elif (shape(hnd) == [5,3,3,2] ):
      ball = True
   return ball

In [9]:
def notrump(hnd):
   NT = False
   if ( balanced(hnd) ):
      if (hcp(hnd) >= 15 and hcp(hnd) <= 23 ):
         NT = True
   if ( len(hnd.spades) == 2 and hcp(hnd.spades) < 3):
      NT = False
   elif(len(hnd.hearts) == 2 and hcp(hnd.hearts) < 3):
      NT = False
   elif(len(hnd.diamonds) == 2 and hcp(hnd.diamonds) < 3):
      NT = False
   elif(len(hnd.clubs) == 2 and hcp(hnd.clubs) < 3):
      NT = False 
   return NT

In [10]:
def strong_club(hnd):
   sc = False
   if ( points_rs(hnd) >= 17 ):
      sc = True
   return sc

In [11]:
def standard(hnd):
   std = False; std_pts = False;
   if ( points_rs(hnd) >= 12 and points_rs(hnd) <= 16 ):
      std_pts = True
   if ( std_pts and rule20(hnd) ):
      std = True
   return std

In [12]:
def twosuiter(hnd):
   ts = False; t = shape(hand)
   if ( t[0] >= 5 and t[1] >= 4 ):
      ts = True
   return ts

In [13]:
d2 = generate_deal('hcp(north) >= 15 && shape(north, any 4xxx) # + 65xx + any 75xx + any 7xxx + any 8xxx)')
hand = d2.north
hand.pprint()
print('Total points =' , hcp(hand), ' or RS points = ', points_rs(hand))
print('   The hand is a Standard Opener: ', standard(hand))
print('   The hand is a 2-suiter: ', twosuiter(hand))
print('       The distributional points: ', dist_points(hand))
print('       The RS distributional points: ', points_rs(hand) - hcp(hand))
print('           The hand is a NoTrump opener: ', notrump(hand))
make_bid_table(hand)

♠ KT2
♥ AQ85
♦ K5
♣ AQJ7
Total points = 19  or RS points =  19
   The hand is a Standard Opener:  False
   The hand is a 2-suiter:  False
       The distributional points:  1
       The RS distributional points:  0
           The hand is a NoTrump opener:  True


Suit,Length,HCP,Controls,Bid Quality
Spades,3,3,0,3.3
Hearts,4,6,0,4.6
Diamonds,2,3,2,2.3
Clubs,4,7,0,4.7


In [14]:
h2 = Hand('KT87.AQ987.KJ95.')
t = shape(h2)
t

[5, 4, 4, 0]

In [15]:
t[0]

5